# Generative Spaces (ABM)

In this workshop we will lwarn how to construct a ABM (Agent Based Model) with spatial behaviours, that is capable of configuring the space. This file is a simplified version of Generative Spatial Agent Based Models. For further information, you can find more advanced versions here:

* [Object Oriented version](https://github.com/shervinazadi/spatial_computing_workshops/blob/master/notebooks/w3_generative_spaces.ipynb)
* [Vectorized version](https://topogenesis.readthedocs.io/notebooks/random_walker)

## 0. Initialization

### 0.1. Load required libraries

In [ ]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
np.random.seed(0)

### 0.2. Define the Neighborhood (Stencil)

In [ ]:
# creating neighborhood definition
stencil = tg.create_stencil("von_neumann", 1, 1)
# setting the center to zero
stencil.set_index([0,0,0], 0)
print(stencil)

### 0.3 Visualize the Stencil

In [ ]:
# initiating the plotter
p = pv.Plotter(notebook=True)

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape because we want to inject our values
grid.dimensions = np.array(stencil.shape) + 1
# The bottom left corner of the data set
grid.origin = [0,0,0]
# These are the cell sizes along each axis
grid.spacing = [1,1,1]

# Add the data values to the cell data
grid.cell_arrays["values"] = stencil.flatten(order="F")  # Flatten the stencil
threshed = grid.threshold([0.9, 1.1])

# adding the voxels: light red
p.add_mesh(threshed, show_edges=True, color="#ff8fa3", opacity=0.3)

# plotting
p.show(use_ipyvtk=True)

## 1. Setup the Environment

### 1.1. Load the envelope lattice as the avialbility lattice

In [ ]:
# loading the lattice from csv
lattice_path = os.path.relpath('../data/voxelized_envelope.csv')
avail_lattice = tg.lattice_from_csv(lattice_path)

### 1.2. Load the Sun Access lattice as the value field

In [ ]:
# loading the lattice from csv
lattice_path = os.path.relpath('../data/sun_access.csv')
sun_acc_lattice = tg.lattice_from_csv(lattice_path)

### 1.3. Initialize the Agents

In [ ]:
# Finding the index of the available voxels in avail_lattice
avail_flat = avail_lattice.flatten()
# avail_flat_index = np.where(avail_flat == 1)[0]
avail_index = np.array(np.where(avail_lattice == 1)).T

# Randomly choosing five available voxels
agn_num = 5
# select_id = np.random.choice(avail_flat_index, agn_num)
select_id = np.random.choice(len(avail_index), agn_num)
agn_origins = avail_index[select_id]

# adding the origins to the agents locations
agn_locs = []
for a_origin in agn_origins:
    agn_locs.append([a_origin])

# # creating the agent ids lis
# agn_ids = list(range(1,agn_num+1))

## 2. ABM Simulation (Agent Based Space Occupation)

In [ ]:
# initialzing the list of frames
frames = [avail_lattice]
# setting the time variable to 0
t = 0
# main feedback loop of the simulation (for each time step ...)
while t<3:
    # for each agent ... 
    for a_id in range(agn_num):
        # retrieve the list of the locations of the current agent
        a_locs = agn_locs[a_id]
        # initialize the list of free neighbours
        free_neighs = []
        # for each location of the agent
        for loc in a_locs:
            # retrieve the list of neighbours of the agent based on the stencil
            neighs = avail_lattice.find_neighbours_masked(stencil, loc = loc)
            
            # for each neighbour ... 
            for n in neighs:
                # if the neighbour is available... 
                if avail_flat[n]:
                    # add the neighbour to the list of free neighbours
                    free_neighs.append(n)

        # check if found any free neighbour
        if len(free_neighs)>0:            
            # convert free neighbours to a numpy array
            free_neighs = np.array(free_neighs)
            # randomly select one of the available neighbours
            selected_neigh_id = np.random.choice(free_neighs, 1)
            # find the location of the newly selected neighbour
            selected_neigh_loc = np.array(np.unravel_index(selected_neigh_id, avail_lattice.shape)).flatten()
            # add the newly selected neighbour location to agent locations
            agn_locs[a_id].append(selected_neigh_loc)
            # set the newly selected neighbour as UNavailable (0) in the availability lattice
            avail_flat[selected_neigh_id] = 0
    
    # constructing the new lattice
    new_avail_lattice = tg.to_lattice(avail_flat, avail_lattice)
    # adding the new lattice to the list of frames
    frames.append(new_avail_lattice)
    # adding one to the time counter
    t += 1

### Credits

In [ ]:
__author__ = "Shervin Azadi "
__license__ = "MIT"
__version__ = "1.0"
__url__ = "https://github.com/shervinazadi/spatial_computing_workshops"
__summary__ = "Spatial Computing Design Studio Workshop on Agent Based Models for Generative Spaces"